<a href="https://colab.research.google.com/github/deonGuo/shroom_oracle/blob/main/shroom_oracle_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

# Use model
path = Path()
shroom_oracle = load_learner(path/'oracle.pkl', cpu=True)
shroom_oracle.dls.vocab

# UI
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description='Pray to Shroom Oracle')
out_pl = widgets.Output()

def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  prediction,prediction_idx,probs = shroom_oracle.predict(img)
  lbl_prediction.value = f'Predictin: {prediction}; Probability: {probs[prediction_idx]}'

btn_run.on_click(on_click_classify)
lbl_prediction = widgets.Label()

VBox([widgets.Label('Behold, O The Great Shroom Oracle'), btn_upload, btn_run, out_pl, lbl_prediction])